### Feed mixed input - trial notebook

1. Load model thru checkpoints, config & mint config utils
    - Similar to `evaluator.py`
1. Load tfrecord dataset as a starting point
1. Mix up inputs in the dataset to create a new input (or new dataset, whichever is easier)
1. Pass mixed input to model and see what it comes up with
1. Prepend original input motion sequence and visualize prediction vs targets
    - (targets being all the un-mixed inputs used to make the mixed input)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from mint.core import inputs
from mint.core import model_builder
from mint.ctl import single_task_evaluator
from mint.utils import config_util
from third_party.tf_models import orbit
import tensorflow as tf

import ipywidgets as widgets
from IPython.display import display

import pprint
import hashlib
import pickle
import numpy as np
import copy
import vedo
from scipy.spatial.transform import Rotation as R
from smplx import SMPL
import time
import torch

embedWindow(verbose=True): could not load ipyvtklink try:
> pip install ipyvtklink


In [3]:
# layout helpers

layout_path_input = widgets.Layout(width='700px', height='40px')

# input widgets

wg_config_path = widgets.Text(
    value="./configs/motion_enc_pilot-audioseed-37_bsz8.config",
    placeholder="Path to config file",
    layout=layout_path_input,
)

wg_checkpoint_dir = widgets.Text(
    value="/srv/share4/anarayanan68/mint/_expts/motion_enc_pilot_bsz8_1GPU/checkpoints",
    placeholder="Checkpoint directory to restore model from",
    layout=layout_path_input,
)

wg_enc_pkl_path = widgets.Text(
    value="/srv/share4/anarayanan68/mint/_expts/motion_enc_pilot_bsz8_1GPU/enc_data.pkl",
    placeholder="Path to pkl file for motion name encoding",
    layout=layout_path_input,
)

wg_name_enc_yaml_path = widgets.Text(
    value="./configs/joint_optim_pilot-name_enc.yml",
    placeholder="Path to name encoder YAML config file",
    layout=layout_path_input,
)

# overall container

wg_container = widgets.VBox([
    widgets.HBox([
        widgets.Label("Path to config file:"),
        wg_config_path,
    ]),
    widgets.HBox([
        widgets.Label("Checkpoint dir:"),
        wg_checkpoint_dir,
    ]),
    widgets.HBox([
        widgets.Label("Path to encoding pkl file:"),
        wg_enc_pkl_path,
    ]),
    widgets.HBox([
        widgets.Label("Path to name enc YAML config file:"),
        wg_name_enc_yaml_path,
    ]),
])

display(wg_container)

In [22]:
wg_config_path.value, wg_checkpoint_dir.value, wg_enc_pkl_path.value, wg_name_enc_yaml_path.value

('./configs/joint_optim_pilot-audioseed37_bsz8.config',
 './_expts/joint_optim_pilot_bsz8_1GPU/checkpoints',
 './_expts/joint_optim_pilot_bsz8_1GPU/enc_data.pkl',
 './configs/joint_optim_pilot-name_enc.yml')

In [5]:
# Config read

configs = config_util.get_configs_from_pipeline_file(wg_config_path.value)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [6]:
name_enc_config_yaml = config_util.read_yaml_config(wg_name_enc_yaml_path.value)

In [7]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    name_encoder_config_yaml=name_enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=wg_checkpoint_dir.value,
    max_to_keep=None)

checkpoint_path = checkpoint_manager.restore_or_initialize()

if checkpoint_path is not None:
    print(f"restored model from {checkpoint_path}.")
else:
    print("initialized model.")

restored model from ./_expts/joint_optim_pilot_bsz8_1GPU/checkpoints/ckpt-27001.


In [8]:
# Load original dataset
orig_dataset = inputs.create_input(
      train_eval_config=eval_config,
      dataset_config=eval_dataset_config,
      is_training=False,
      use_tpu=False,
      overfit_expt=True)

In [9]:
# see samples in dataset
for i,x in enumerate(orig_dataset):
    if i >= 2:
        break
    print(i, list(x.keys()))
    pprint.pprint(x)

0 ['motion_name_enc', 'audio_name', 'audio_sequence_shape', 'motion_name', 'motion_sequence_shape', 'target', 'actual_motion_input', 'audio_input']
{'actual_motion_input': <tf.Tensor: shape=(1, 120, 225), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.5071713e-03, -1.2368782e-03,  9.9999809e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.6000727e-03, -1.2885372e-03,  9.9999791e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -2.0099527e-03, -1.2556252e-03,  9.9999720e-01],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -4.9028394e-04, -5.5003497e-03,  9.9998474e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -4.2695715e-04, -5.1946845e-03,  9.9998641e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          7.4367061e-05, -4.8195366e-03,  9.9998838e-01]]], dtype=float32)>,
 'audio_input': <tf.T

**Each entry in the `orig_dataset` is the `inputs.py`-preprocessed dict of each `tfexample` written by `tools/preprocessing.py`**
<br>
So to make a new input, just create a dict that can be passed to the model just as `SingleTaskEvaluator` passes its `inputs`. No `tf` dataset necessary.
<br>
But, to compute the input, use some inputs from the `orig_dataset` and also the encoding scheme from `tools/preprocessing.py`. Also don't forget to use the actual motion inputs etc from there.

In [10]:
# Fns from preprocessing, for motion name encoding

from tools import preprocessing

In [24]:
enc_pkl_data = preprocessing.load_enc_pkl(wg_enc_pkl_path.value)
enc_pkl_data

{'onehot_enc_map': {'len': 20,
  'gWA_sFM_cAll_d25_mWA4_ch05': {'index': 0,
   'enc': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d25_mWA2_ch03': {'index': 1,
   'enc': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d27_mWA2_ch21': {'index': 2,
   'enc': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d25_mWA5_ch07': {'index': 3,
   'enc': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d27_mWA2_ch17': {'index': 4,
   'enc': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d25_mWA5_ch06': {'index': 5,
   'enc': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.])},
  'gWA_sFM_cAll_d26_mWA5_ch13': {'index

In [12]:
it = iter(orig_dataset)
inp1 = next(it)
inp2 = next(it)

pprint.pprint(inp1)

{'actual_motion_input': <tf.Tensor: shape=(1, 120, 225), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.5071713e-03, -1.2368782e-03,  9.9999809e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -1.6000727e-03, -1.2885372e-03,  9.9999791e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -2.0099527e-03, -1.2556252e-03,  9.9999720e-01],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -4.9028394e-04, -5.5003497e-03,  9.9998474e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -4.2695715e-04, -5.1946845e-03,  9.9998641e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          7.4367061e-05, -4.8195366e-03,  9.9998838e-01]]], dtype=float32)>,
 'audio_input': <tf.Tensor: shape=(1, 240, 35), dtype=float32, numpy=
array([[[ -114.73737 ,  -246.10489 , -2078.7415  , ..., -1105.4794  ,
         -2209.8015  , -1484.

In [13]:
pprint.pprint(inp2)

{'actual_motion_input': <tf.Tensor: shape=(1, 120, 225), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          7.3011895e-04, -3.3532334e-03,  9.9999410e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          7.8092772e-04, -3.3680149e-03,  9.9999404e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          9.4373431e-04, -3.4757780e-03,  9.9999350e-01],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          8.2552433e-04, -3.0448267e-03,  9.9999505e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          6.3653680e-04, -2.8562604e-03,  9.9999571e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          2.7560751e-04, -2.6339253e-03,  9.9999648e-01]]], dtype=float32)>,
 'audio_input': <tf.Tensor: shape=(1, 240, 35), dtype=float32, numpy=
array([[[ 228.6918   ,  177.93639  , -530.1248   , ..., -154.09326  ,
         -580.7614   , -300.6

In [14]:
# Sanity check: passing actual input and saving output
op_inp1 = model(inp1)
np.save("./_expts/op_inp1.npy", op_inp1)

In [15]:
op_inp2 = model(inp2)
np.save("./_expts/op_inp2.npy", op_inp2)

In [16]:
display(inp1['target'].shape)
display(inp1['audio_name'], inp1['motion_name'])

TensorShape([1, 60, 225])

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'mLH2'], dtype=object)>

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'gLH_sBM_cAll_d18_mLH2_ch06'], dtype=object)>

In [17]:
display(inp2['target'].shape)
display(inp2['audio_name'], inp2['motion_name'])

TensorShape([1, 60, 225])

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'mWA5'], dtype=object)>

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'gWA_sFM_cAll_d25_mWA5_ch07'], dtype=object)>

**Expt 1: Midpt *after* encoding** - NOT POSSIBLE (would need to modify the model to do this now)

**Expt 2: Midpt *before* encoding**

In [19]:
name1, name2 = bytes.decode(inp1['motion_name'].numpy().item()), bytes.decode(inp2['motion_name'].numpy().item())
display(name1, name2, type(name1))

'gLH_sBM_cAll_d18_mLH2_ch06'

'gWA_sFM_cAll_d25_mWA5_ch07'

str

In [25]:
name_latent_1, name_latent_2 = (
    preprocessing.get_latent_from_seq_name(name1, enc_pkl_data),
    preprocessing.get_latent_from_seq_name(name2, enc_pkl_data)
)
name_latent_1, name_latent_2

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [26]:
mix_latent = 0.5 * (name_latent_1 + name_latent_2)
mix_latent

array([0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0.5, 0. , 0. , 0. ])

In [29]:
mix_inp__before_enc = copy.deepcopy(inp1)
mix_inp__before_enc['motion_name_enc'] = mix_latent[None, ...]  # add batch dim

op = model(mix_inp__before_enc)
np.save("./_expts/mix_op__before_enc.npy", op)